In [1]:
from collections import Counter
import pandas
import json
import regex as re
from typing import Generator

texts = pandas.read_csv("./pc-texts.csv")
#texts.drop(labels=["Auteur réel si différent", "Source attrib (lien, doi, bibl)", "Auteur", "Source"], axis=1,
#           inplace=True)

texts = texts[texts.tokens > 1000]


with open("stopwords.json") as f:
    STOPWORDS = json.load(f)

#KEEP = ('TYPOGRAPHICAL SYMBOLS', 'SINGLE LETTERS (LATIN)', 'SINGLE LETTERS (GREEK)', 'GREEK NUMERALS (1-100)',
#        "ARABIC NUMERALS (0-100)", 'ROMAN NUMERALS (1-100)', 'CRITICAL ABBREVIATIONS', 'ARTICLES', 'PARTICLES',
#        'INTERJECTIONS', 'CONJUNCTIONS', 'PREPOSITIONS/POSTPOSITIONS', 'ADVERBS', 'PRONOUNS', 'NOUNS', 'ADJECTIVES',
#        'VERBS')

KEEP = ('ARTICLES', 'PARTICLES', 'INTERJECTIONS', 'CONJUNCTIONS', 'PREPOSITIONS/POSTPOSITIONS', 'ADVERBS', 'PRONOUNS',
        'ADJECTIVES', 'VERBS')


def recursive_stopwords(sw_dict: dict) -> Generator[str, str, None]:
    for key, value in sw_dict.items():
        if isinstance(value, list):
            yield from value
        else:
            yield from recursive_stopwords(value)


STOPWORDS = set(recursive_stopwords({k: v for k, v in STOPWORDS.items() if k in KEEP}))


print(f"{len(STOPWORDS)} found.")


def get_function_words(string: str) -> Counter:
    string = re.sub(r"[[:punct:]]", " ", string.replace("ʹ", "'"))
    words = [w for w in string.split() if w in STOPWORDS]
    return Counter(words)


total = Counter()
FWORDS_DF = []

for idx, text in texts.iterrows():
    fwords = get_function_words(text.text)
    total += fwords
    FWORDS_DF.append({**text.to_dict(), **fwords})

FWORDS_DF = pandas.DataFrame(FWORDS_DF)
del FWORDS_DF["text"]

FWORDS_DF.fillna(0).to_csv("pc-function-words.csv", index=False)

6109 found.
